### Data-driven optimization and decision making

### Final project

Juha Reinikainen

In [15]:
import pandas as pd
import numpy as np
from statistics import NormalDist

In [16]:
df = pd.read_csv("data/ADSDE.csv")
#Date,Open,High,Low,Close,Adj Close,Volume
# df.hist("Close")
x = df.get("Close").interpolate()
x = x.to_numpy()
 


x0 = x[0]
xReturn = (x - x0) / x0
norm = NormalDist(mu = xReturn.mean(), sigma = xReturn.std())

expectedReturn = sum([norm.pdf(x) * x for x in xReturn])
print(xReturn)
print(expectedReturn)

[ 0.          0.04643918  0.03742859  0.02789817  0.02200661 -0.00502506
  0.07901583  0.09721026  0.10362164  0.08092187  0.10136892  0.06082141
  0.07624329  0.10656732  0.04661242  0.05441002  0.03638884  0.03638884
 -0.02148668 -0.02269966 -0.06342051 -0.09097211 -0.04903828 -0.03916128
 -0.01819442  0.03413622 -0.02755149 -0.02148668 -0.1219892  -0.13567838
 -0.11696408 -0.12424187 -0.12701437 -0.12250907 -0.14243629 -0.1254548
 -0.13151961 -0.18159763 -0.16790845 -0.18142434 -0.20499046 -0.25888056
 -0.35193205 -0.29301678 -0.24935015 -0.26373244 -0.27187657 -0.28054058
 -0.30341359 -0.27898107 -0.32864318 -0.34770401 -0.37314155 -0.3985791 ]
-8.00339127732982
